### Imports

In [56]:
from ibm_watson_studio_lib import access_project_or_space
from urllib.parse import urlencode
import time
import requests

import urllib3
urllib3.disable_warnings()

### Setup

In [ ]:
wslib = access_project_or_space()

CPD_HOST = "cpd-cpd.company.com"
PROJECT_ID = wslib.here.get_ID()
TOKEN = wslib.auth.get_current_token()

### Main API Call

In [ ]:
def update_dq_rule(token, project_id, rule_id):
    """
    Update a single data quality rule using the Data Quality API
    
    Args:
        token (str): Authorization token
        project_id (str): The identifier of the project
        rule_id (str): The data quality rule identifier
    
    Returns:
        dict: Response with success status and details
    """
    try:
        
        # Build URL
        url = f'https://{CPD_HOST}/data_quality/v3/projects/{project_id}/rules/{rule_id}'
        
        # Set headers
        headers = {
            'Authorization': f'Bearer {token}',
            'Content-Type': 'application/json-patch+json'
        }

        # Set body
        body = [
                  {
                    'op': 'replace',
                    'path': '/output',
                    'value': {
                        'database': {
                            'location': {
                                'connection': {
                                    'id': 'acb03b96-1d60-40bc-aae3-46ab56071832'
                                },
                                'schema_name': 'BIDEMODATA',
                                'table_name': 'DQ_OUTPUT_SQL'
                            },
                            'records_type': 'failing_records',
                            'update_type': 'append'
                        },
                        'maximum_record_count': 5,
                        'columns': [
                            {
                                'name': 'Failing_rules',
                                'type': 'metric',
                                'metric': 'failing_rules'
                            },
                            {
                                'name': 'Job_ID',
                                'type': 'metric',
                                'metric': 'job_id'
                            },
                            {
                                'name': 'Job_run_ID',
                                'type': 'metric',
                                'metric': 'job_run_id'
                            },
                            {
                                'name': 'Passing_rules',
                                'type': 'metric',
                                'metric': 'passing_rules'
                            },
                            {
                                'name': 'Percent_failing_rules',
                                'type': 'metric',
                                'metric': 'percent_failing_rules'
                            },
                            {
                                'name': 'Percent_passing_rules',
                                'type': 'metric',
                                'metric': 'percent_passing_rules'
                            },
                            {
                                'name': 'Project_ID',
                                'type': 'metric',
                                'metric': 'project_id'
                            },
                            {
                                'name': 'Record_ID',
                                'type': 'metric',
                                'metric': 'record_id'
                            },
                            {
                                'name': 'Rule_ID',
                                'type': 'metric',
                                'metric': 'rule_id'
                            },
                            {
                                'name': 'Rule_name',
                                'type': 'metric',
                                'metric': 'rule_name'
                            },
                            {
                                'name': 'System_date',
                                'type': 'metric',
                                'metric': 'system_date'
                            },
                            {
                                'name': 'System_time',
                                'type': 'metric',
                                'metric': 'system_time'
                            }
                        ],
                        'inherit_project_level_output_setting': False,
                        'create_table_only_when_issues_are_found': False,
                        'import_table_in_project': True
                    }
                  }
                ]
        
        # Make the PATCH request
        response = requests.patch(url, headers=headers, json=body, verify=False)
        
        if response.status_code == 200:
            # Success
            return {
                'success': True,
                'message': f'Data quality rule {rule_id} updated successfully',
                'rule_id': rule_id
            }
            
        elif response.status_code == 401:
            return {
                'success': False,
                'error': 'Unauthorized - Authorization missing, invalid, or expired',
                'status_code': response.status_code,
                'rule_id': rule_id
            }
            
        elif response.status_code == 403:
            return {
                'success': False,
                'error': 'Forbidden - No permission to update the data quality rule',
                'status_code': response.status_code,
                'rule_id': rule_id
            }
            
        elif response.status_code == 404:
            return {
                'success': False,
                'error': 'Not Found - Data quality rule cannot be found',
                'status_code': response.status_code,
                'rule_id': rule_id
            }
            
        elif response.status_code == 500:
            return {
                'success': False,
                'error': 'Internal Server Error - Data quality rule cannot be updated',
                'status_code': response.status_code,
                'rule_id': rule_id,
                'response': response.text
            }
            
        else:
            return {
                'success': False,
                'error': f"Unexpected HTTP {response.status_code}",
                'status_code': response.status_code,
                'rule_id': rule_id,
                'response': response.text
            }
            
    except ValueError as ve:
        return {
            'success': False,
            'error': f"Validation error: {str(ve)}",
            'rule_id': rule_id
        }
    except requests.exceptions.RequestException as re:
        return {
            'success': False,
            'error': f"Request error: {str(re)}",
            'rule_id': rule_id
        }
    except Exception as e:
        return {
            'success': False,
            'error': f"Unexpected error: {str(e)}",
            'rule_id': rule_id
        }

### Main Logic

In [ ]:
# List all assets of type Data Quality Rule
dq_rules = wslib.assets.list_assets('data_rule')

if len(dq_rules) == 0:
    print('No rules to update')
else:
    print(f"\nUpdating {len(dq_rules)} data quality rules...")
    
    success_count = 0
    error_count = 0
    
    # Iterate through the rules
    for i, rule in enumerate(dq_rules, 1):
        rule_id = rule['asset_id']
        rule_name = rule['name']
        
        print(f"Updating rule {i}/{len(dq_rules)}: {rule_id} - {rule_name}")
        
        # Call the main function
        result = update_dq_rule(
            token=TOKEN,
            project_id=PROJECT_ID,
            rule_id=rule_id
        )
        
        # Print results
        if result['success']:
            success_count += 1
            print(f'  ✓ Successfully updated')
        else:
            error_count += 1
            print(f"  ✗ Failed: {result['error']}")
        
        # Small delay between calls
        if i < len(dq_rules):
            time.sleep(0.2)
    
    print(f"\nCompleted!")
    print(f"  Successful: {success_count}")
    print(f"  Errors: {error_count}")